# Video: Structuring Tables for Joins in Pandas

This video describes how to use and manage data frame indexes for efficient joins in pandas.

## Incoming Data to Their Spec

* Often data comes structured differently than what you need...
  * Sometimes you can push for a change upstream.
  * Sometimes you need to handle it as is.

## The Original Garden Bed Materials Data

![Screenshot off Google Sheet showing calculation of material requirements](https://raw.githubusercontent.com/bu-cds-omds/dx602-examples/main/data/garden-bed_size_materials-original.png)

Script:
* The original spreadsheet for the material requirements looked like this.
* Each row was a different size, and the material needs were calculated.
* Note the formula at the top of the screenshot.
* Formats like this are convenient for this manual planning, but not so convenient for joins.
* So if you get the data like this, what can you do?

## Code Example: Reshaping Spreadsheet Data

In [ ]:
import pandas as pd

In [ ]:
bed_size_materials_original = pd.read_csv("https://raw.githubusercontent.com/bu-cds-omds/dx602-examples/main/data/garden-bed_size_materials-original.tsv", sep="\t")
bed_size_materials_original

,Bed Width (ft),Bed Length (ft),Height Target (in),Bottom Target (in),Bottom Pieces,Bottom Width (in),Height Pieces,"2"" x 6"" x 4'","2"" x 6"" x 8'","8"" x 8"" x 16"" Cinder Block"
0,4,4,18,45,8,44,3,20,0,12
1,4,8,18,45,8,44,3,6,14,24


Script:
* This file was produced by downloading the spreadsheet.
* Let's add on the bed size column that we previously used.
* It can be calculated from the width and length columns after converting them to string.


In [ ]:
bed_size_materials_original["bed_size"] = bed_size_materials_original["Bed Width (ft)"].astype("string") + "' x " + bed_size_materials_original["Bed Length (ft)"].astype('string') + "'"
bed_size_materials_original

,Bed Width (ft),Bed Length (ft),Height Target (in),Bottom Target (in),Bottom Pieces,Bottom Width (in),Height Pieces,"2"" x 6"" x 4'","2"" x 6"" x 8'","8"" x 8"" x 16"" Cinder Block",bed_size
0,4,4,18,45,8,44,3,20,0,12,4' x 4'
1,4,8,18,45,8,44,3,6,14,24,4' x 8'


Script:
* Besides those string conversions, it looks like just adding strings together normally in Python.
* The next transformation is a little bit tricky.
* We're going to make a new data frame that takes the bed size column, one of the three material columns, and makes a new row from each of those pairs.


In [ ]:
bed_size_materials = bed_size_materials_original.melt(
    id_vars=["bed_size"],
    value_vars=[c for c in bed_size_materials_original.columns if " x " in c],
    var_name="material",
    value_name="quantity_per_bed")
bed_size_materials

,bed_size,material,quantity_per_bed
0,4' x 4',"2"" x 6"" x 4'",20
1,4' x 8',"2"" x 6"" x 4'",6
2,4' x 4',"2"" x 6"" x 8'",0
3,4' x 8',"2"" x 6"" x 8'",14
4,4' x 4',"8"" x 8"" x 16"" Cinder Block",12
5,4' x 8',"8"" x 8"" x 16"" Cinder Block",24


Script:
* Take a minute to match up the column names of this data frame with the inputs to the melt method.
* And try to match up the values between this new data frame and the previous one.
* You may find it easier to open the notebook accompanying this video so you can scroll up and down and compare.
* PAUSE
* Now material is a column, and contains the name of one of the three material columns in the previous data frame.


In [ ]:
for m in bed_size_materials["material"].unique():
    print(m)

2" x 6" x 4'
2" x 6" x 8'
8" x 8" x 16" Cinder Block


Script:
* And the new quantity per bed column refers to the value from the material column in the original data frame.
* So we end up with a row for each pair of bed size and material.
* Next I will get the material data like before.
* I'll go through this part more quickly since it is a repeat.

In [ ]:
material_costs = pd.read_csv(
    "https://raw.githubusercontent.com/bu-cds-omds/dx602-examples/main/data/garden-material_costs.tsv",
    sep="\t",
    index_col="material")
material_costs

,unit_cost
material,
"2"" x 6"" x 4'",4.92
"2"" x 6"" x 8'",6.62
"8"" x 8"" x 16"" Cinder Block",2.53


In [ ]:
bed_size_costs = bed_size_materials.join(material_costs, on="material")
bed_size_costs

,bed_size,material,quantity_per_bed,unit_cost
0,4' x 4',"2"" x 6"" x 4'",20,4.92
1,4' x 8',"2"" x 6"" x 4'",6,4.92
2,4' x 4',"2"" x 6"" x 8'",0,6.62
3,4' x 8',"2"" x 6"" x 8'",14,6.62
4,4' x 4',"8"" x 8"" x 16"" Cinder Block",12,2.53
5,4' x 8',"8"" x 8"" x 16"" Cinder Block",24,2.53


In [ ]:
bed_size_costs["cost"] = bed_size_costs["quantity_per_bed"] * bed_size_costs["unit_cost"]
bed_size_costs

,bed_size,material,quantity_per_bed,unit_cost,cost
0,4' x 4',"2"" x 6"" x 4'",20,4.92,98.40
1,4' x 8',"2"" x 6"" x 4'",6,4.92,29.52
2,4' x 4',"2"" x 6"" x 8'",0,6.62,0.00
3,4' x 8',"2"" x 6"" x 8'",14,6.62,92.68
4,4' x 4',"8"" x 8"" x 16"" Cinder Block",12,2.53,30.36
5,4' x 8',"8"" x 8"" x 16"" Cinder Block",24,2.53,60.72


In [ ]:
bed_size_costs = bed_size_costs.groupby("bed_size")[["cost"]].sum()
bed_size_costs

,cost
bed_size,
4' x 4',128.76
4' x 8',182.92


Script:
* And there we are at the previous result, despite the different format of the original table.
* What if we need to reorganize our data in the opposite direction?

In [ ]:
bed_size_materials.pivot_table(
    index="bed_size",
    columns="material",
    values="quantity_per_bed"
)

material,"2"" x 6"" x 4'","2"" x 6"" x 8'","8"" x 8"" x 16"" Cinder Block"
bed_size,,,
4' x 4',20,0,12
4' x 8',6,14,24


Script:
* The pivot_table method will let you reorganize rows into a table.
* If you have ever used pivot tables in Microsoft Excel or any other spreadsheet program, it is pretty similar.
* There's a similar method called pivot.
* Those two functions differ in aggregation and non-numeric value support.
* Read the documentation for details.
* Melt and pivot table are probably the reshaping functions you will use the most.
* There are a few others.
* The resources section of this lesson has a link to them, so take a look so you are aware of what other options are available.
